In [ ]:
# Mount Google Drive to save files directly if needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install transformers if not already installed
!pip install transformers==4.25.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.25.1 which is incompatible.


In [ ]:
# Import necessary libraries
import json
import torch
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm.auto import tqdm
import os

In [ ]:
# Disable unnecessary parallel tokenizers logging if preferred
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Configuration class for model and dataset paths
class CFG:
    INPUT = '/content/drive/MyDrive/dataset/Bengali hate speech(1) .csv'  # Update this path as needed
    TRANS_MODEL = 'facebook/nllb-200-distilled-600M'

In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/dataset/Bengali hate speech(1) .csv')
display(df.head())

,sentence,hate,category
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports


In [ ]:
# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(CFG.TRANS_MODEL)
model.to(device)
model.eval()  # Set model to evaluation mode
tokenizer = AutoTokenizer.from_pretrained(CFG.TRANS_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [ ]:
# Save model and tokenizer (optional)
torch.save(model.state_dict(), '/content/drive/MyDrive/nllb-200-distilled-600M.pth')
tokenizer.save_pretrained('/content/drive/MyDrive/tokenizer/')

('/content/drive/MyDrive/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/tokenizer/sentencepiece.bpe.model',
 '/content/drive/MyDrive/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/tokenizer/tokenizer.json')

In [ ]:
# Define the translation function
def translate_by_nllb(text, tokenizer, model, device):
    # Tokenize the text and move inputs to the GPU
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate translation
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"],
        max_length=64
    )
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

In [ ]:
# Create a new DataFrame to store translated sentences with the 'sentence' and 'hate' columns
translated_df = pd.DataFrame(columns=['sentence', 'labels'])

# Apply translation to each row within the specified range (15 to 30)
for i, row in tqdm(df.iterrows(), total=len(df)):
    if 15 <= i <= 30:
        translated_sentence = translate_by_nllb(row['sentence'], tokenizer, model, device)
        # Create a temporary DataFrame for the new row
        new_row = pd.DataFrame({'sentence': [translated_sentence], 'labels': [row['hate']]})
        # Concatenate the new row with the existing DataFrame
        translated_df = pd.concat([translated_df, new_row], ignore_index=True)

# Display the first few rows of the new DataFrame with translated sentences
display(translated_df.head(9))

  0%|          | 0/30000 [00:00<?, ?it/s]

,sentence,labels
0,I'm not sure.,1
1,I don't want to get the shoes off the bet. I c...,1
2,The stomach of the son of Papan Maghir is full...,1
3,It is necessary to give the shoes to your sins,1
4,I would have fucked the guy with the jersey.,1
5,Yeshir Ali has no idea how to play,1
6,You son of a bitch.,1
7,The polarity of the Papan Magir is doing it.,1
8,The name of the company,1


In [ ]:
translated_df.shape

(16, 2)

In [ ]:
translated_df .to_csv('hello.csv', index=False)